In [ ]:
# @title Step 1: Install Required Libraries (for Google Gemini)
# This installs the libraries for Google Gemini, Google Search, reading Word documents, and handling images.
!pip install -q "google-generativeai" "google-search-results" python-docx Pillow

print("✅ All required libraries for Google Gemini have been installed!")


In [ ]:
# @title Step 2: Enter Your API Keys
import os
from getpass import getpass
import google.generativeai as genai

# Securely prompt for the Google AI API key
try:
  GOOGLE_API_KEY = getpass('Enter your Google AI Studio API key: ')
  genai.configure(api_key=GOOGLE_API_KEY)
  print("🔑 Google Gemini API Key has been set.")
except Exception as e:
  print(f"Could not set Google API key: {e}")


# Securely prompt for the SerpApi API key
try:
  SERPAPI_API_KEY = getpass('Enter your SerpApi API key: ')
  os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY
  print("🔑 SerpApi API Key has been set.")
except Exception as e:
  print(f"Could not set SerpApi API key: {e}")

In [ ]:
# @title Step 3: Upload Your Document and Image Files
from google.colab import files

print("Please upload your .docx analysis file and your .png/.jpg graph image.")
uploaded = files.upload()

# Confirm which files were uploaded
print("\n--- Uploaded Files ---")
for filename in uploaded.keys():
    print(f"📄 {filename}")
print("--------------------")
print("\n✅ Files uploaded successfully. Please copy the exact filenames for the next step.")

In [ ]:
# @title Step 4: Define the Analysis Functions (Using the High-Quota 'Flash' Model)
import base64
import docx
from serpapi import GoogleSearch
from IPython.display import display, Markdown
import google.generativeai as genai
import PIL.Image

# --- HELPER FUNCTIONS ---

def extract_text_from_docx(file_path):
    try:
        doc = docx.Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs if para.text.strip() != ""])
    except Exception as e:
        print(f"Error reading docx file: {e}")
        return f"Error reading docx file: {e}"

def fetch_real_time_data(queries: list):
    print("Fetching real-time data from the web...")
    all_snippets = []
    for query in queries:
        try:
            params = {"engine": "google", "q": query, "api_key": os.environ['SERPAPI_API_KEY']}
            search = GoogleSearch(params)
            results = search.get_dict()
            snippet = results.get('organic_results', [{}])[0].get('snippet', 'No snippet found.')
            all_snippets.append(f"- {query}: {snippet}")
            print(f"  - Found info for: {query}")
        except Exception as e:
            all_snippets.append(f"- Could not fetch info for '{query}': {e}")
    return "\n".join(all_snippets)


# --- REWRITTEN FUNCTION FOR GEMINI ---

def generate_vlm_analysis(team_name, doc_text, image_paths: list, external_knowledge):
    """
    Calls Google Gemini with text, MULTIPLE images, and real-time data to generate insights.
    """
    print("\nSending data to the Google Gemini Model for analysis...")

    # --- ⬇️⬇️ THIS IS THE ONLY LINE THAT CHANGED ⬇️⬇️ ---
    # We are now using the 'flash' model, which has a much more generous free rate limit.
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    # --- ⬆️⬆️ END OF CHANGE ⬆️⬆️ ---

    prompt_content = f"""
You are an expert digital strategist analyzing social media content for the IPL franchise: {team_name}.
Your analysis should be a detailed, natural-language narrative. Synthesize all the information provided below.
*1. Document-Based Analysis:*
{doc_text}
---
*2. Real-Time Performance Data (IPL 2025):*
{external_knowledge}
---
*3. Visual Data Analysis (Multiple Images):*
The attached images contain key data visualizations. Please analyze all of them.
*Your Task:*
Synthesize all three sources to provide clear, strategic content recommendations. Structure your output with clear headings, tables for keywords, and bullet points.
"""

    input_for_gemini = [prompt_content]
    print("Opening and preparing images for Gemini...")
    for image_path in image_paths:
        try:
            img = PIL.Image.open(image_path)
            input_for_gemini.append(img)
            print(f"  - Image '{image_path}' prepared.")
        except Exception as e:
            print(f"  - Could not open or process image '{image_path}': {e}")

    try:
        response = model.generate_content(input_for_gemini)
        print("✅ Analysis received from the Gemini Model!")
        return response.text
    except Exception as e:
        print(f"\n--- AN ERROR OCCURRED ---")
        print(f"The call to the Gemini API failed. Error: {e}")
        return "The analysis could not be generated due to an API error."


print("✅ All functions are now defined and ready to use (Google Gemini 'Flash' model enabled).")

In [ ]:
# @title ✏️ Step 5: Configure and Run Your Analysis!

# --- 1. SET YOUR INPUTS HERE ---
# (Make sure the filenames match what you uploaded in Step 3)
team_to_analyze = "DC"
document_path = "DC_data_doc.docx"

# ⬇️⬇️⬇️  ADD ALL YOUR IMAGE FILENAMES TO THIS LIST  ⬇️⬇️⬇️
image_data_paths = [
    "image(1).png",
    "image(2)).png", # Example: add your second image file name
    "image(3).png", # Example: add your third image file name
    "image(4).png"
]
# ⬆️⬆️⬆️  ENSURE THE FILENAMES ABOVE ARE CORRECT  ⬆️⬆️⬆️

# (Customize these search queries for the team's key players/topics)
real_time_queries = [
    f"KL Rahul performance in IPL 2025",
    f"Axar patel captaincy in IPL 2025",
    f"{team_to_analyze} team news 2025"
]

# --- 2. THE SCRIPT WILL RUN FROM HERE ---
print(f"🚀 Starting analysis for {team_to_analyze} with {len(image_data_paths)} images...")

# Extract text from the local document
document_content = extract_text_from_docx(document_path)

# Fetch fresh information from the internet
real_time_content = fetch_real_time_data(real_time_queries)

# Call the VLM with all the context to generate the final analysis
# Note: We now pass the list image_data_paths to the function
final_insights = generate_vlm_analysis(
    team_name=team_to_analyze,
    doc_text=document_content,
    image_paths=image_data_paths, # Passing the list of image paths
    external_knowledge=real_time_content
)

# --- 3. SAVE THE OUTPUT TO A FILE ---
report_filename = f"{team_to_analyze}_strategy_report.md"
with open(report_filename, "w") as f:
    f.write(final_insights)

print(f"\n✨ Success! Your report is ready and saved as '{report_filename}'.")
print("You can view the formatted output in the next cell.")

In [ ]:
# @title 📊 Step 6: View and Download the Generated Report

# Display the report with Markdown formatting for better readability
print("--- GENERATED STRATEGIC INSIGHTS ---")
display(Markdown(final_insights))

# Provide a link to download the saved file
from google.colab import files
print(f"\n\n--- Download Your Report ---")
print(f"Click the link below to download the report file '{report_filename}'.")
files.download(report_filename)